In [1]:
import random
puzzle1 = [
	[4,1,3],
	[2,8,0],
	[7,6,5]
]
puzzle2 = [
	[10, 3, 6, 4],
	[ 1, 5, 8, 0],
	[ 2,13, 7,15],
	[14, 9,12,11]
]
puzzle3 = [
	[ 3, 7,14,15,10],
	[ 1, 0, 5, 9, 4],
	[16, 2,11,12, 8],
	[17, 6,13,18,20],
	[21,22,23,19,24]
]

UP, DOWN = 0, 1
LEFT, RIGHT = 2, 3
END = 5
MOVE_TYPES = [LEFT,RIGHT,UP,DOWN,END]

In [2]:
def getGridFitness(grid):
    lastNumber = 0
    n = len(grid)
    for row in range(n):
        for col in range(n):
            num = grid[row][col]
            #Empty tile doesn't affect fitness
            if(num == 0):
                continue
            #Fitness search ends when a mismatch is found
            if(lastNumber+1 != num):
                return lastNumber
            lastNumber = num
    #This is a winning solution
    return lastNumber
solvedPuzzle1 = [[1,2,3],[4,5,6],[7,8,0]]
print("Expected fitness of 8, got", getGridFitness(solvedPuzzle1))
solvedPuzzle2 = [[1,2,3],[4,5,6],[7,0,8]]
print("Expected fitness of 8, got", getGridFitness(solvedPuzzle2))
first3Correct = [[1,2,3],[8,8,8],[8,8,8]]
print("Expected fitness of 3, got", getGridFitness(first3Correct))

Expected fitness of 8, got 8
Expected fitness of 8, got 8
Expected fitness of 3, got 3


In [3]:
def applyMoves(grid, startRow, startCol, moves):
    '''
    Avoid refinding the 0 by only looking once, and then providing it going forward.
    '''    
    row = startRow
    col = startCol
    if(grid[row][col] != 0):
        raise Exception("FUCKER")
    n = len(grid)
    for move in moves:
        if(move == END):
            return grid, row, col
        if(move == LEFT and col > 0):
            grid[row][col-1], grid[row][col] = grid[row][col], grid[row][col-1]
            col -= 1
            continue 
        if(move == RIGHT and col < n-1):
            grid[row][col+1], grid[row][col] = grid[row][col], grid[row][col+1]
            col += 1
            continue  
        if(move == UP and row > 0):
            grid[row-1][col], grid[row][col] = grid[row][col], grid[row-1][col]
            row -= 1
            continue 
        if(move == DOWN and row < n-1):
            grid[row+1][col], grid[row][col] = grid[row][col], grid[row+1][col]
            row += 1
            continue  
        #getting here the moves are invalid
        break
    return grid, row, col
print(applyMoves([[1,2,3],[4,5,6],[7,8,0]],2,2,[LEFT,LEFT,UP,UP,RIGHT,RIGHT,DOWN, DOWN]))
#expect [[2, 3, 6], [1, 5, 8], [4, 7, 0]]

([[2, 3, 6], [1, 5, 8], [4, 7, 0]], 2, 2)


In [4]:
def revertGrid(grid, startRow, startCol, moves):
    '''
    Is it always faster to reverse a series of moves, than 
    to simply clone the original grid? It depends on the length of the 
    average solution relative to n
    '''
    reverseMoves = []
    for move in moves:
        if(move == END):
            break
        reverseMoves.append(move)
    return applyMoves(grid, startRow, startCol, reverseMoves[::-1])
revertGrid([[1,2,3],[4,5,6],[7,8,0]],2,2,[LEFT, UP, RIGHT, LEFT, END, LEFT])
#expect [[1, 2, 3], [4, 0, 5], [7, 8, 6]]

([[1, 2, 3], [4, 0, 5], [7, 8, 6]], 1, 1)

In [5]:
def mutateMoveList(moves, mutationChance = 0.05):
    #No mutation
    if(random.random() > mutationChance):
        return
    mutate_index = random.randint(0, len(moves)-1)
    moves[mutate_index] = MOVE_TYPES[random.randint(0, 4)]
        
ms = [0,1]
mutateMoveList(ms,1.0)

In [6]:
def crossoverMoves(moveSet1, moveSet2):
    i = random.randint(0, len(moveSet1)-1)
    return moveSet1[:i]+moveSet2[i:], moveSet2[:i]+moveSet1[i:]

In [7]:
def generateRandomMoveSet(targetLen):
    return [MOVE_TYPES[random.randint(0, 4)] for _ in range(targetLen)]

def generatePopulation(popSize, targetLen):
    return [generateRandomMoveSet(targetLen) for _ in range(popSize)]
generatePopulation(4, 20)

[[1, 0, 0, 0, 5, 2, 3, 2, 5, 1, 1, 3, 3, 5, 2, 1, 3, 3, 2, 5],
 [1, 1, 5, 2, 2, 2, 1, 3, 3, 2, 3, 0, 5, 2, 2, 1, 0, 3, 0, 3],
 [3, 2, 5, 5, 2, 3, 5, 3, 3, 5, 0, 0, 0, 0, 1, 3, 0, 5, 5, 1],
 [5, 3, 5, 0, 1, 1, 3, 0, 1, 1, 0, 2, 0, 3, 1, 0, 0, 2, 3, 0]]

In [8]:
def selectChromeForCrossover(priorGen, totalFitness):
    '''
    Select and return a move-set, based on its fitnesses.
    Where a fitness of 0 stands almost no chance of being selected.
    The total fitness gets provided, so that it is only calculated once, outside of the method
    '''
    index = random.randint(0, len(priorGen)-1)
    while(True):        
        totalFitness -= priorGen[index][1]
        if(totalFitness <= 0):
            break
        index = random.randint(0, len(priorGen)-1)
    return priorGen[index][0]

In [9]:
def getNewGeneration(priorGen):
    '''
    priorGen - A list of tuples, where the tuples are (moves, fitness)
    The new generation will have just as many chromosomes, but will
    be based on crossovers and mutations of the previous generations (i.e priorGen)
    '''
    totalFitness = 0
    for chrome in priorGen:
        totalFitness += chrome[1]
    print("Average fitness:", totalFitness/len(priorGen))
    ret = []
    while(len(ret) < len(priorGen)):
        c1 = selectChromeForCrossover(priorGen, totalFitness)
        c2 = selectChromeForCrossover(priorGen, totalFitness)
        crossed = crossoverMoves(c1, c2)
        mutateMoveList(crossed[0])
        mutateMoveList(crossed[1])
        ret.append(crossed[0])
        ret.append(crossed[1])
    return ret
getNewGeneration([([1,2,3,4],2),
                  ([5,6,7,8],200),
                  ([9,10,11,12],2000),
                  ([13,14,15,16],2000000)])

Average fitness: 500550.5


[[13, 14, 15, 16], [13, 14, 15, 16], [13, 14, 15, 12], [9, 10, 11, 16]]

In [10]:
puzzle1 = [
	[4,1,3],
	[2,8,0],
	[7,6,5]
]
max_fitness = 0
for f in puzzle1:
    max_fitness += sum(f)

r = 1
c = 2
max_generations = 30
core_puzzle = puzzle1
populationSize = 2000
currentPopulation = generatePopulation(populationSize, 50)
for i in range(max_generations):
    fitnesses = []
    #evaluation the current generations fitness
    for moves in currentPopulation:
        result = applyMoves([x[:] for x in core_puzzle], r, c, moves)
        fitness = getGridFitness(result[0])
        if(fitness == f):
            print("SOLUTION", moves)
            break
        fitnesses.append((moves, fitness))

    #create a new generation based on the content of fitnesses
    currentPopulation = getNewGeneration(fitnesses)
print("DONE")

Average fitness: 0.012
Average fitness: 0.9705
Average fitness: 0.982
Average fitness: 0.998
Average fitness: 1.0805
Average fitness: 1.364
Average fitness: 2.0235
Average fitness: 3.0335
Average fitness: 3.6715
Average fitness: 3.859
Average fitness: 3.8995
Average fitness: 3.9325
Average fitness: 3.938
Average fitness: 3.9315
Average fitness: 3.9465
Average fitness: 3.9465
Average fitness: 3.9405
Average fitness: 3.9365
Average fitness: 3.933
Average fitness: 3.949
Average fitness: 3.949
Average fitness: 3.957
Average fitness: 3.971
Average fitness: 3.973
Average fitness: 3.954
Average fitness: 3.9535
Average fitness: 3.965
Average fitness: 3.9565
Average fitness: 3.9535
Average fitness: 3.955
DONE
